<a href="https://colab.research.google.com/github/cwils021/Census-Data-Wrangling/blob/main/2016Census_data_wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2016 StatCan Census Data (Age, Gender of Population of Canada, Provinces and Territories, Census Divisions, Census Subdivisions and Dissemination Areas

## Requirements before running Script

1. Download CSV data from StatCan [Age Data Table](https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/dt-td/Rp-eng.cfm?TABID=4&LANG=E&A=R&APATH=3&DETAIL=0&DIM=0&FL=A&FREE=0&GC=01&GL=-1&GID=1159582&GK=1&GRP=1&O=D&PID=109526&PRID=10&PTYPE=109445&S=0&SHOWALL=0&SUB=0&Temporal=2016&THEME=115&VID=0&VNAMEE=&VNAMEF=&D1=0&D2=0&D3=0&D4=0&D5=0&D6=0)
2. Clean up column names to match what script will expect (see table below) (ex. ensure no whitespace, drop empty columns (notes)) - I did this in Excel quickly while examining file

|                     Old Column Name                     |  New Column Name  |
| :-----------------------------------------------------: | :---------------: |
|                       CENSUS_YEAR                       |    CENSUS_YEAR    |
|                     GEO_CODE (POR)                      |     GEO_CODE      |
|                        GEO_LEVEL                        |     GEO_LEVEL     |
|                        GEO_NAME                         |     GEO_NAME      |
|                           GNR                           |        GNR        |
|                    DATA_QUALITY_FLAG                    | DATA_QUALITY_FLAG |
|                      CSD_TYPE_NAME                      |   CSD_TYPE_NAME   |
|                      ALT_GEO_CODE                       |   ALT_GEO_CODE    |
|    DIM: Age (in single years) and average  age (127)    |      AGE_CAT      |
| Member ID: Age (in single years) and  average age (127) |    AGE_CAT_ID     |
|   Notes: Age (in single years) and average  age (127)   |   **DROP COLUMN**   |
|        Dim: Sex (3): Member ID: [1]: Total - Sex        |       TOTAL       |
|           Dim: Sex (3): Member ID: [2]: Male            |    TOTAL_MALE     |
|          Dim: Sex (3): Member ID: [3]: Female           |   TOTAL_FEMALE    |



## Long to Wide Script

### Load File as DataFrame

In [1]:
# imports
from google.colab import files
import pandas as pd
from functools import reduce

# Need to Implement Chunking for full dataset to speed up file upload

raw_data = files.upload()
filename = str(list(raw_data.keys())[0])

Saving test_full.csv to test_full.csv


In [2]:
raw_df = pd.read_csv(filename)
# sanity check
raw_df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CENSUS_YEAR,GEO_CODE,GEO_LEVEL,GEO_NAME,GNR,DATA_QUALITY_FLAG,CSD_TYPE_NAME,ALT_GEO_CODE,AGE_CAT,AGE_CAT_ID,TOTAL,TOTAL_MALE,TOTAL_FEMALE
0,2016,1,0,Canada,4.0,20000,,1,Total - Age,1,35151730.0,17264200.0,17887530.0
1,2016,1,0,Canada,4.0,20000,,1,0 to 14 years,2,5839570.0,2992920.0,2846645.0
2,2016,1,0,Canada,4.0,20000,,1,0 to 4 years,3,1898790.0,973030.0,925755.0
3,2016,1,0,Canada,4.0,20000,,1,Under 1 year,4,369730.0,189085.0,180650.0
4,2016,1,0,Canada,4.0,20000,,1,1,5,372615.0,190905.0,181715.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2016,24350053,4,24350053,3.7,0,,24350150053,45,59,5.0,0.0,5.0
1048571,2016,24350053,4,24350053,3.7,0,,24350150053,46,60,10.0,5.0,0.0
1048572,2016,24350053,4,24350053,3.7,0,,24350150053,47,61,0.0,0.0,0.0
1048573,2016,24350053,4,24350053,3.7,0,,24350150053,48,62,10.0,5.0,0.0


### Define WideDataRow Class

In [3]:
class WideDataRow:
  def __init__(self, long_data_row):
    self.year = long_data_row['CENSUS_YEAR']
    self.geo_code = long_data_row['GEO_CODE']
    self.geo_level = long_data_row['GEO_LEVEL']
    self.geo_name = long_data_row['GEO_NAME']
    self.gnr = long_data_row['GNR']
    self.data_quality = long_data_row['DATA_QUALITY_FLAG']
    self.csd_type = long_data_row['CSD_TYPE_NAME']
    self.alt_geo_code = long_data_row['ALT_GEO_CODE']
    self.age_groups_total = {}
    self.age_groups_male = {}
    self.age_groups_female = {}

  def add_age_data(self, long_data_row):
    self.age_groups_total[long_data_row['AGE_CAT']] = long_data_row['TOTAL']
    self.age_groups_male[long_data_row['AGE_CAT'] + '_male'] = long_data_row['TOTAL_MALE']
    self.age_groups_female[long_data_row['AGE_CAT'] + '_female'] = long_data_row['TOTAL_FEMALE']

### Define a Age Group Reference Dict

In [4]:
age_group_key_ref = {
   1: 'Total - Age',
   3: '0 to 4 years',
   9: '5 to 9 years',
  15: '10 to 14 years',
  22: '15 to 19 years',
  28: '20 to 24 years',
  34: '25 to 29 years',
  40: '30 to 34 years',
  46: '35 to 39 years',
  52: '40 to 44 years',
  58: '45 to 49 years',
  64: '50 to 54 years',
  70: '55 to 59 years',
  76: '60 to 64 years',
  83: '65 to 69 years',
  89: '70 to 74 years',
  95: '75 to 79 years',
 101: '80 to 84 years',
 108: '85 to 89 years',
 114: '90 to 94 years',
 120: '95 to 99 years',
 126: '100 years and over'
}

### Iterate Over Raw Data Rows with parse_data()

Here we define the function parse data, which iterates over the rows of raw data creating WideDataRow objects. It returns a dict of WideDataRow Objects

In [5]:
def parse_data(raw_df, ref_dict):
  
  wide_data = {}

  for i, row in raw_df.iterrows():  
    if row['AGE_CAT_ID'] in ref_dict:
      if row['GEO_CODE'] not in wide_data:
        wide_data_obj = WideDataRow(row)
        wide_data_obj.add_age_data(row)
        wide_data[row['GEO_CODE']] = wide_data_obj
      else:
        wide_data[row['GEO_CODE']].add_age_data(row)
      
  
  return wide_data

### Return Wide Format of Age Population Data

The function wide_rows iterates over the values of the dictionary of parsed data calling the obj_to_row function on each row object and returns the concatenated rows

In [6]:
def obj_to_row(wide_obj):
  meta = pd.Series({
                   'year': wide_obj.year,
                   'geo_code':wide_obj.geo_code,
                   'geo_name':wide_obj.geo_name,
                   'gnr':wide_obj.gnr,
                   'data_quality':wide_obj.data_quality,
                   'csd_type':wide_obj.csd_type,
                   'alt_geo_code':wide_obj.alt_geo_code
                   }).to_frame().transpose()


  total_ages = pd.Series(wide_obj.age_groups_total,dtype=int).to_frame().transpose()
  male_ages = pd.Series(wide_obj.age_groups_male, dtype=int).to_frame().transpose()
  female_ages = pd.Series(wide_obj.age_groups_female, dtype=int).to_frame().transpose()

  series_list = [meta, total_ages, male_ages, female_ages]

  combined = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True ), series_list)
  return combined

In [7]:
def wide_rows(parsed_data):
 
 wide_rows = [obj_to_row(row_obj) for row_obj in parsed_data]

 return pd.concat(wide_rows).reset_index()

# sanity check
wide_rows(parse_data(raw_df, age_group_key_ref).values())



,index,year,geo_code,geo_name,gnr,data_quality,csd_type,alt_geo_code,Total - Age,0 to 4 years,5 to 9 years,10 to 14 years,15 to 19 years,20 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,45 to 49 years,50 to 54 years,55 to 59 years,60 to 64 years,65 to 69 years,70 to 74 years,75 to 79 years,80 to 84 years,85 to 89 years,90 to 94 years,95 to 99 years,100 years and over,Total - Age_male,0 to 4 years_male,5 to 9 years_male,10 to 14 years_male,15 to 19 years_male,20 to 24 years_male,25 to 29 years_male,30 to 34 years_male,35 to 39 years_male,40 to 44 years_male,45 to 49 years_male,50 to 54 years_male,55 to 59 years_male,60 to 64 years_male,65 to 69 years_male,70 to 74 years_male,75 to 79 years_male,80 to 84 years_male,85 to 89 years_male,90 to 94 years_male,95 to 99 years_male,100 years and over_male,Total - Age_female,0 to 4 years_female,5 to 9 years_female,10 to 14 years_female,15 to 19 years_female,20 to 24 years_female,25 to 29 years_female,30 to 34 years_female,35 to 39 years_female,40 to 44 years_female,45 to 49 years_female,50 to 54 years_female,55 to 59 years_female,60 to 64 years_female,65 to 69 years_female,70 to 74 years_female,75 to 79 years_female,80 to 84 years_female,85 to 89 years_female,90 to 94 years_female,95 to 99 years_female,100 years and over_female
0,0,2016,1,Canada,4,20000,,1,35151730,1898790,2018130,1922650,2026155,2242695,2285990,2329395,2288370,2255135,2359960,2678070.0,2620240.0,2290515.0,1972475.0,1420875.0,1021850.0,749645.0,482520.0,223505.0,56525.0,8230.0,17264200,973030,1034685,985200,1039215,1144495,1144470,1148290,1118635,1104440,1157760,1318750.0,1285185.0,1114885.0,953075.0,677975.0,469545.0,325765.0,185535.0,68675.0,13245.0,1340.0,17887530,925755,983445,937445,986945,1098200,1141520,1181110,1169730,1150695,1202210,1359320.0,1335055.0,1175630.0,1019405.0,742900.0,552300.0,423885.0,296985.0,154835.0,43280.0,6895.0
1,0,2016,10,Newfoundland and Labrador,4,0,,10,519715,22360,26040,26035,27255,27705,28250,29400,30740,34510,38665,42620.0,43080.0,42030.0,37485.0,26165.0,16945.0,11065.0,6175.0,2485.0,610.0,95.0,253925,11435,13435,13335,13980,13920,14095,14185,14550,16745,18745,20730.0,21035.0,20565.0,18310.0,12725.0,8015.0,4890.0,2300.0,785.0,135.0,15.0,265790,10930,12610,12700,13275,13785,14150,15215,16190,17760,19920,21890.0,22045.0,21460.0,19180.0,13445.0,8930.0,6175.0,3875.0,1700.0,475.0,75.0
2,0,2016,1001,Division No. 1,3.7,0,,1001,270345,12530,14260,13835,14150,16665,17600,17650,17690,18455,19495,21420.0,20930.0,19625.0,17285.0,11995.0,7555.0,4910.0,2735.0,1220.0,300.0,55.0,131475,6390,7280,7095,7260,8240,8825,8600,8495,9015,9385,10305.0,10135.0,9475.0,8265.0,5765.0,3530.0,2070.0,915.0,360.0,65.0,10.0,138870,6135,6980,6745,6895,8425,8770,9050,9195,9440,10120,11115.0,10795.0,10150.0,9015.0,6230.0,4020.0,2840.0,1815.0,860.0,230.0,40.0
3,0,2016,1001105,"Portugal Cove South, T",1.7,900,T,1001105,150,0,0,0,5,10,5,5,0,0,10,20.0,30.0,25.0,10.0,10.0,0.0,5.0,0.0,0.0,0.0,0.0,70,0,0,0,0,5,5,0,0,0,5,5.0,15.0,15.0,0.0,10.0,0.0,5.0,0.0,0.0,0.0,0.0,75,0,0,5,10,5,5,5,0,0,5,10.0,15.0,10.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0
4,0,2016,10010734,10010734,1.7,900,,10011050734,150,0,0,0,10,10,10,5,0,0,10,15.0,30.0,25.0,10.0,15.0,0.0,5.0,0.0,0.0,0.0,0.0,75,0,0,5,0,5,0,0,0,5,5,10.0,15.0,15.0,5.0,10.0,0.0,5.0,0.0,0.0,0.0,0.0,80,0,0,0,5,5,5,0,0,0,5,10.0,15.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8252,0,2016,24350027,24350027,4.6,0,,24350100027,390,15,10,10,15,10,15,15,10,20,15,20.0,50.0,50.0,50.0,35.0,20.0,20.0,15.0,0.0,0.0,0.0,200,5,5,5,10,5,5,5,5,10,10,10.0,20.0,25.0,30.0,20.0,5.0,10.0,5.0,0.0,0.0,0.0,190,10,5,5,5,5,5,5,10,10,5,15.0,25.0,25.0,20.0,20.0,10.0,10.0,5.0,0.0,0.0,0.0
8253,0,2016,24350028,243500